# EDA on retail Data

Online retail refers to the buying and selling of goods and services over the internet, allowing consumers to browse, select, and purchase products through digital platforms. The importance of customers in online retail is paramount, as they serve as the lifeblood of this industry. In the digital realm, customer satisfaction, loyalty, and engagement play crucial roles in driving sales and fostering a sustainable business model. Positive online experiences, personalized interactions, and efficient customer service are pivotal elements that not only attract new customers but also retain existing ones. The ability to understand and cater to customer preferences, analyze purchasing patterns, and adapt strategies accordingly distinguishes successful online retailers, as customer-centric approaches directly impact brand reputation and long-term success in the competitive landscape of e-commerce.

In [26]:
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import numpy as np 
import seaborn as sns



%matplotlib inline

# Handling data

In [27]:
data = pd.read_csv("../online_retail.csv")
data.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom


Our data includes 8 columns:
1.	InvoiceNo: label/number of deal. ( 53628 unique values recorded.)
2.	StockCode: represent the Stock name/Code. Unique value for each stock. (5305 unique values recorded)
3.	Description: unknown and ambiguous
4.	Quantity: numbers of the stock unit
5.	InvoiceDate: Data of Deal (Data is from the start of 2010 and until the end of 2011)
6.	UnitPrice: price of each unit of data
7.	CustomerID: represents the id of each user. Unique value for each user (5943 Values recorded)
8.	Country: probably location of user


In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1590484 entries, 0 to 1590483
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   InvoiceNo    1590484 non-null  object 
 1   StockCode    1590484 non-null  object 
 2   Description  1584658 non-null  object 
 3   Quantity     1590484 non-null  int64  
 4   InvoiceDate  1590484 non-null  object 
 5   UnitPrice    1590484 non-null  float64
 6   CustomerID   1218722 non-null  float64
 7   Country      1590484 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 97.1+ MB


In [29]:
data["InvoiceDate"] = pd.to_datetime(data["InvoiceDate"])

datatype of InvoiceDate - CustomerID got modified 

# Data Cleaning

In [30]:
data.isna().sum()

InvoiceNo           0
StockCode           0
Description      5826
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     371762
Country             0
dtype: int64

In [31]:
data.loc[(data["Description"].isna() == True) & (data["CustomerID"].isna() == False)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


When the 'Description' column is NaN, only rows where the 'CustomerID' is also NaN.

In [32]:
data.dropna(subset=['CustomerID'], inplace=True)
data.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom


Removed 371,762 rows that did not have a valid CustomerID. dataset has no Null value

In [33]:
data.drop_duplicates(inplace=True)

Identified and removed numerous duplicate rows.

# Data Investigation

In [34]:
data.head(3) 

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom


In the upcoming three sections, I will address the following topics:

1. Calculating the total cash flow for each country along with the number of deals per country.

2. Analyzing the total cash flow based on stock.

3. Evaluating whether the data is suitable for RFM (Recency, Frequency, Monetary) analysis.


In [35]:
data["total_cash"] = data["Quantity"] * data["UnitPrice"]
data.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,total_cash
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00


## Cash - Country

In [49]:
condition = {"CustomerID": "count",
             "total_cash" : "sum"}
country_data = data.groupby(by=["Country", data["InvoiceDate"].dt.year]).agg(condition).reset_index()
country_data.columns = ["country", "year", "buyers", "total_cash"]
country_data.head()

,country,year,buyers,total_cash
0,Australia,2009,27,50.25
1,Australia,2010,639,30322.00
2,Australia,2011,1224,136072.17
3,Austria,2009,38,1998.34
4,Austria,2010,508,11281.98


In [71]:
index_to_drop = country_data.index[country_data["total_cash"] < 0].tolist()
index_to_drop

[]

In [72]:
country_data.drop(country_data.index[index_to_drop], inplace=True)

In [84]:
fig = px.treemap(
    country_data,
    path=[px.Constant("all"), "country", "year"],
    values='total_cash',
    #root_color="lightgrey",
    title="amount of money for each country",
    # margin=dict(t=50, l=25, r=25, b=25),
    width=1100, 
    height=400
)

fig.show()

In [90]:
condition = {"CustomerID": "count",
             "total_cash" : "sum"}
stock_data = data.groupby(by=["StockCode", data["InvoiceDate"].dt.year]).agg(condition).reset_index()
stock_data.columns = ["stock", "year", "buyers", "total_cash"]
stock_data.head()

,stock,year,buyers,total_cash
0,10002,2009,17,180.20
1,10002,2010,259,5065.37
2,10002,2011,33,509.15
3,10080,2009,3,3.40
4,10080,2010,3,6.80


### removing outliers

In [107]:
stock_data = stock_data.loc[~ ((stock_data["total_cash"] > 50000) | (stock_data["buyers"] > 1500) | (stock_data["total_cash"] < 0))]

stock_data["year"] = stock_data["year"].astype(str)
stock_data.head(3)

,stock,year,buyers,total_cash
0,10002,2009,17,180.20
1,10002,2010,259,5065.37
2,10002,2011,33,509.15


In [122]:
fig = px.scatter(stock_data, 
                 x="buyers", 
                 y="total_cash", 
                 color="year",  
                 facet_col="year",
                 title="Classify the total money on each product annually.")
fig.show()

# RFM Analysis

RFM analysis, a widely used marketing technique, relies on three crucial parameters: **Recency, Frequency, and Monetary Value**. To assess the suitability of our database for RFM analysis, we'll examine its features and characteristics.

RFM analysis categorizes customers based on Recency, Frequency, and Monetary Value. It helps businesses identify and prioritize customer segments, enabling targeted marketing strategies. This approach enhances customer engagement, satisfaction, and retention.


1. **Recency:**
   
   The dataset encompasses user interactions recorded during the years 2009, 2010, and 2011. Within this timeframe, numerous deals were captured, allowing us to examine the most recent interactions users had with our system.

2. **Frequency:**
   
   The dataset reveals a significant variation in interaction frequency. The highest number of deals for a user is 12,638, while the lowest is just one. This broad spectrum of interaction frequencies provides valuable insights into user engagement with the system.

3. **Monetary Value:**
   
   Calculating the total value each user contributed to the system involves multiplying the unit price by the quantity of items purchased and summing them up. This monetary analysis allows us to gauge the financial impact of each user on the system.

In a separate notebook, we have performed RFM (Recency, Frequency, Monetary) Analysis using this dataset to gain deeper insights into user behavior and system engagement.

# conclusion

**Strengths:**
1. Abundant parameters are meticulously recorded, contributing to the normalcy and comprehensiveness of our dataset.
2. The data is well-documented, encompassing nearly all essential factors for a thorough analysis.
3. the data is very suitable to create a RFM model

**Weaknesses:**
1. Limited to the time frame of 2009-2011, indicating a need for an update to capture more recent trends and behaviors.
2. The constrained time frame hinders the implementation of time-series analysis, limiting our ability to capture and analyze temporal patterns and trends.
3. The data points for the year 2009 are notably sparse. 
